In [38]:
pip install s3fs

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [39]:
pip install kafka-python


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [40]:
import pandas as pd
from kafka import KafkaProducer, KafkaConsumer
from time import sleep
from json import dumps
import json

In [41]:
from cred import ipaddr
producer=KafkaProducer(bootstrap_servers=[ipaddr+':9092'],value_serializer=lambda x: dumps(x).encode('utf-8'))

In [42]:
# producer.send('demotest', value={'hello11':'w1o1rld'})
# producer.send('demotest', value={'n1ame1':'gok1u1l'})
# producer.send('<topic_name>', value=dict_stock)

In [43]:
df=pd.read_csv('indexProcessed.csv')

In [44]:

df.head()

,Index,Date,Open,High,Low,Close,Adj Close,Volume,CloseUSD
0,HSI,1986-12-31,2568.300049,2568.300049,2568.300049,2568.300049,2568.300049,0.0,333.879006
1,HSI,1987-01-02,2540.100098,2540.100098,2540.100098,2540.100098,2540.100098,0.0,330.213013
2,HSI,1987-01-05,2552.399902,2552.399902,2552.399902,2552.399902,2552.399902,0.0,331.811987
3,HSI,1987-01-06,2583.899902,2583.899902,2583.899902,2583.899902,2583.899902,0.0,335.906987
4,HSI,1987-01-07,2607.100098,2607.100098,2607.100098,2607.100098,2607.100098,0.0,338.923013


In [45]:
df.sample(1).to_dict()

{'Index': {50093: 'N225'},
 'Date': {50093: '2002-03-22'},
 'Open': {50093: 11460.9502},
 'High': {50093: 11519.67969},
 'Low': {50093: 11326.21973},
 'Close': {50093: 11345.08008},
 'Adj Close': {50093: 11345.08008},
 'Volume': {50093: 0.0},
 'CloseUSD': {50093: 113.4508008}}

In [46]:
# df.sample(1).to_dict()
# if we dont do orient='records' then it will be in dict format and not json format
# also we will see some extra characters like {0:} in the json, which is not required
# so we need to do orient='records' to remove that so that it is in json format
df.sample(1).to_dict(orient='records')[0]
# this is the format we want to send to kafka



{'Index': 'NYA',
 'Date': '1968-12-03',
 'Open': 645.950012,
 'High': 645.950012,
 'Low': 645.950012,
 'Close': 645.950012,
 'Adj Close': 645.950012,
 'Volume': 0.0,
 'CloseUSD': 645.950012}

In [47]:
import time
while True:
    dict_stock=df.sample(1).to_dict(orient='records')[0]
    producer.send('demotest', value=dict_stock)
    # to not overload the server and see the results realtime, lets use time delay
    time.sleep(1)

In [ ]:
producer.flush()
# to remove all the messages from the queue